# Hugging Face Large Model Inference

This notebook demonstrates how to deploy Hugging Face transformer models using Hugging Face Text Generation Inference (TGI) Deep Learning Container on Amazon SageMaker.

TGI is an open source, high performance inference library that can be used to deploy large language models from Hugging Face’s repository in minutes. The library includes advanced functionality like model parallelism and dynamic batching to simplify production inference with large language models like flan-t5-xxl, LLaMa, StableLM, and GPT-NeoX. 

## Setup

### Install the SageMaker Python SDK

First, make sure that the latest version of SageMaker SDK is installed.

In [1]:
%pip install "sagemaker>=2.163.0"

Note: you may need to restart the kernel to use updated packages.


### Setup account and role

Then, we import the SageMaker python SDK and instantiate a `sagemaker_session` which we use to determine the current region and execution role.

In [2]:
import sagemaker
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
import time

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/22/25 16:04:46] INFO     Found credentials from IAM Role:                                   ]8;id=256543;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=49241;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


[03/22/25 16:04:47] INFO     Found credentials from IAM Role:                                   ]8;id=594257;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=942677;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=584062;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=783772;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

## Retrieve the LLM Image URI

We use the helper function `get_huggingface_llm_image_uri()` to generate the appropriate image URI for the Hugging Face Large Language Model (LLM) inference.

The function takes a required parameter `backend` and several optional parameters. The `backend` specifies the type of backend to use for the model, the values can be "lmi" and "huggingface". The "lmi" stands for SageMaker LMI inference backend, and "huggingface" refers to using Hugging Face TGI inference backend.

In [3]:
image_uri = get_huggingface_llm_image_uri(
  backend="huggingface", # or lmi
  region=region
)

[03/22/25 16:04:59] INFO     Defaulting to only available Python version: py311                   ]8;id=361532;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=770060;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#610\610]8;;\

                    INFO     Defaulting to only supported image scope: gpu.                       ]8;id=649939;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=99166;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#534\534]8;;\

## Create the Hugging Face Model

Next we configure the `model` object by specifying a unique name, the `image_uri` for the managed TGI container, and the execution role for the endpoint. Additionally, we specify a number of environment variables including the `HF_MODEL_ID` which corresponds to the model from the HuggingFace Hub that will be deployed, and the `HF_TASK` which configures the inference task to be performed by the model.

You should also define `SM_NUM_GPUS`, which specifies the tensor parallelism degree of the model. Tensor parallelism can be used to split the model across multiple GPUs, which is necessary when working with LLMs that are too big for a single GPU. Here, you should set `SM_NUM_GPUS` to the number of available GPUs on your selected instance type.

Additionally, we could reduce the memory footprint of the model by setting the `HF_MODEL_QUANTIZE` environment variable to true.

In [4]:
model_name = "falcon-7b-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

hub = {
    'HF_MODEL_ID':'tiiuae/falcon-7b',
    'HF_TASK':'text-generation',
    'SM_NUM_GPUS':'1' # Setting to 1 because sharding is not supported for this model
}

model = HuggingFaceModel(
    name=model_name,
    env=hub,
    role=role,
    image_uri=image_uri
)

[03/22/25 16:05:05] INFO     Found credentials from IAM Role:                                   ]8;id=604062;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=837830;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

## Creating a SageMaker Endpoint

Next we deploy the model by invoking the `deploy()` function.

To efficiently deploy and run large language models, it is important to choose an appropriate instance type that can handle the computational requirements. Here we use an `ml.g5.2xlarge` instance which come with 1 GPU.

Please refer to the [guide](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-choosing-instance-types.html) provided by AWS SageMaker on large model inference instance type selection.

In [5]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge",
    endpoint_name=model_name
)

[03/22/25 16:05:10] INFO     Creating model with name: falcon-7b-2025-03-22-16-05-04                ]8;id=528795;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=308048;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

                    INFO     Creating endpoint-config with name falcon-7b-2025-03-22-16-05-04       ]8;id=443209;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=863210;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5937\5937]8;;\

[03/22/25 16:05:11] INFO     Creating endpoint with name falcon-7b-2025-03-22-16-05-04              ]8;id=878620;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=883472;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4759\4759]8;;\

---------------!

## Running Inference

Once the endpoint is up and running, we can evaluate the model using the `predict()` function.

In [11]:
input_data = {
  "inputs": "The diamondback terrapin was the first reptile to",
  "parameters": {
    "do_sample": True,
    "max_new_tokens": 100,
    "temperature": 0.7,
    "watermark": True
  }
}

predictor.predict(input_data)

[{'generated_text': 'The diamondback terrapin was the first reptile to appear in my yard. I found its tracks on the path between my house and our garage on the evening of January 14, 2011.\nThe second reptile I found in my yard was a juvenile alligator snapping turtle. I found this one swimming in my pond on April 18, 2011.\nThe third reptile I found in my yard was a juvenile alligator snapping turtle. I found this one in my pond on September 13, 2011.\n'}]

## Cleaning Up

After you've finished using the endpoint, it's important to delete it to avoid incurring unnecessary costs.

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()

## Conclusion

In this tutorial, we used a TGI container to deploy Falcon-7B using on a SageMaker `ml.g5.2xlarge` instance. With Hugging Face's Text Generation Inference and SageMaker Hosting, you can easily host large language models like GPT-NeoX, flan-t5-xxl, and LLaMa.